<a href="https://colab.research.google.com/github/Youngkyun-Kwon/Project/blob/main/Optimization_Sustainability_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sustainability Regression Project**
-------------------------------------------------------------------------


## **Summary**
---
The growth of CO2 emissions is a major contributing factor to the speed of climate change. The threat of climate change is growing that will affect the living conditions and environments for human beings on the Earth. In this project, we are going to create a linear regression model to predict **"CO2 emissions, total (KtCO2)"** using the other numerical features in the data set. We are going to show the details as follows:

* Explanatory Data Analysis on this dataset
* Building a linear regression model

The two main goals of this project include:

* Cleaning up and preparing the data in this data set
* Solving an optimization model to find a linear regression model

# Import libs

We will need to install a solver to solve a mixed-integer nonlinear program (!) - we can just run this code to get the `bonmin` solver. More detail here:

* https://projects.coin-or.org/Bonmin/browser/stable/0.1/Bonmin/doc/BONMIN_UsersManual.pdf?format=raw

In [ ]:
# import modules
import pandas as pd
import numpy as np
from pylab import *
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("bonmin") or os.path.isfile("bonmin")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
        !unzip -o -q bonmin-linux64
        # !apt-get install -y -qq mindtpy
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

assert(shutil.which("bonmin") or os.path.isfile("bonmin"))

from pyomo.environ import *

     |████████████████████████████████| 9.2 MB 16.8 MB/s 
     |████████████████████████████████| 49 kB 4.4 MB/s 


# **Read Data**
---

We are going to download the raw data `Data` and data dictionary ` Series` to predict our response variable:

1. `Data`: It contains regional data represented as 58 time series from 1990 to 2011, including GDP, Population, and more.
2. `Series`: These series are divided into seven categories.

In [ ]:
# use pd.read_excel to read both data sets
# https://docs.google.com/spreadsheets/d/1cB3ep8hBXrUcBp-DHoplEpJABaZsaCQ3/edit?usp=sharing&ouid=105133078201960946471&rtpof=true&sd=true
!gdown --id 1cB3ep8hBXrUcBp-DHoplEpJABaZsaCQ3
df_data = pd.read_excel('climate_change_download_0.xls',sheet_name='Data')
df_series = pd.read_excel('climate_change_download_0.xls',sheet_name='Series')

Downloading...
From: https://drive.google.com/uc?id=1cB3ep8hBXrUcBp-DHoplEpJABaZsaCQ3
To: /content/climate_change_download_0.xls
100% 4.86M/4.86M [00:00<00:00, 42.9MB/s]


## **Raw Data**

In [ ]:
# show first few rows from df_data
df_data.head()

,Country code,Country name,Series code,Series name,SCALE,Decimals,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
0,ABW,Aruba,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,29.5748,..,..,..,..,..,..,..,..,..,29.5748,..,..,..,..,..,..,..,..,..,..,..
1,ADO,Andorra,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..
2,AFG,Afghanistan,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..
3,AGO,Angola,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0.208235,..,..,..,..,..,..,..,..,..,0.208235,..,..,..,..,..,..,..,..,..,..,..
4,ALB,Albania,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,4.96788,..,..,..,..,..,..,..,..,..,4.96788,..,..,..,..,..,..,..,..,..,..,..


## **Data Dictionary**

In [ ]:
# show first few rows from df_series
df_series.head()

,Series code,Series name,Scale,Decimals,Order,Topic,Definition,Source
0,SP.POP.TOTL,Population,0,0,1,Size of the economy,Population includes all residents who are pres...,(1) United Nations Population Division. 2011. ...
1,SP.POP.GROW,Population growth (annual %),0,1,2,Size of the economy,Annual population growth rate for year t is th...,Derived from total population. Population sour...
2,NY.GDP.MKTP.CD,GDP ($),0,0,3,Size of the economy,GDP is gross domestic product and measures the...,"World Bank national accounts data, and OECD Na..."
3,NY.GNP.PCAP.CD,GNI per capita (Atlas $),0,0,4,Size of the economy,"GNI per capita is the gross national income, c...","World Bank national accounts data, and OECD Na..."
4,EN.CLC.MMDT.C,"Average daily min/max temperature (1961-1990, ...",Text,Text,5,Climate,Average daily min/max temperature are the mini...,"Mitchell, T.D., Carter, T.R., Jones, P.D., Hul..."


## **Merge Dataframes**

We merged these data sets into one master dataframe by using `Series code` and named it as `df`.

In [ ]:
# combine df_data and df_series into one master dataframe
# use series code to merge both datasets
# the master dataframe will be named as df
# do a merge where `df_data` is left and `df_series  is right
df = pd.merge(df_data, df_series, left_on='Series code', right_on='Series code')
print(df.shape) # print the shape of df
df.info() # check the info
# df contains 13512 and 35 columns after merging

(13512, 35)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 13512 entries, 0 to 13511
Data columns (total 35 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Country code   13512 non-null  object
 1   Country name   13512 non-null  object
 2   Series code    13512 non-null  object
 3   Series name_x  13512 non-null  object
 4   SCALE          13512 non-null  object
 5   Decimals_x     13512 non-null  object
 6   1990           10017 non-null  object
 7   1991           10017 non-null  object
 8   1992           10017 non-null  object
 9   1993           10017 non-null  object
 10  1994           10017 non-null  object
 11  1995           10017 non-null  object
 12  1996           10017 non-null  object
 13  1997           10017 non-null  object
 14  1998           10017 non-null  object
 15  1999           10017 non-null  object
 16  2000           10017 non-null  object
 17  2001           10017 non-null  object
 18  2002          

After merging both datasets, our master dataframe has 13512 rows and 35 columns.

# **Data Preparation**
---

In this section, we are going to consolidate the times series and create a data dictionary.

## **Consolidate Time Series**

Since we have 21 years of data from 1990 to 2011, we are going to replace time series data for a single value representing the average of the non-null entries.

### **Subet Time Series Data**

First, we subset the data from 1990 to 2011 and store in a tempoyary dataframe,`years`.

In [ ]:
# compute the average of the non-null entries as a single value to replace the time series data
year = range(1990, 2012) # subset 21 years of data from 1990 to 2011
test = df[list(year)] # retrive the time series data
test # show the time series data

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
0,29.5748,..,..,..,..,..,..,..,..,..,29.5748,..,..,..,..,..,..,..,..,..,..,..
1,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..
2,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..
3,0.208235,..,..,..,..,..,..,..,..,..,0.208235,..,..,..,..,..,..,..,..,..,..,..
4,4.96788,..,..,..,..,..,..,..,..,..,4.96788,..,..,..,..,..,..,..,..,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13507,2.49718e+06,2.69364e+06,2.90976e+06,3.13964e+06,3.37393e+06,3.60526e+06,3.81758e+06,4.02428e+06,4.22936e+06,4.43979e+06,4.6612e+06,4.89894e+06,5.14862e+06,5.41033e+06,5.68341e+06,5.96746e+06,6.27572e+06,6.59727e+06,6.93279e+06,7.28307e+06,7.6487e+06,..
13508,18304000,1.88649e+07,1.94461e+07,2.00485e+07,2.06729e+07,21320400,2.19921e+07,2.26976e+07,2.34387e+07,2.42174e+07,25036000,2.57692e+07,2.63456e+07,2.69044e+07,2.74482e+07,2.79887e+07,2.85336e+07,2.90798e+07,2.96369e+07,3.01938e+07,3.08446e+07,..
13509,1.01209e+07,1.05695e+07,1.106e+07,1.15686e+07,1.20615e+07,1.25151e+07,1.29906e+07,1.34286e+07,1.3852e+07,1.42965e+07,1.47886e+07,1.54294e+07,1.61253e+07,1.68678e+07,1.76408e+07,1.8432e+07,1.93337e+07,2.02614e+07,2.12165e+07,2.22018e+07,2.322e+07,..
13510,3.09686e+06,3.14167e+06,3.18326e+06,3.22351e+06,3.26494e+06,3.30912e+06,3.35693e+06,3.40748e+06,3.45843e+06,3.50665e+06,3.55014e+06,3.64072e+06,3.72988e+06,3.81964e+06,3.91287e+06,4.01183e+06,4.12899e+06,4.25314e+06,4.38486e+06,4.52456e+06,4.61473e+06,..


### **Check Data Types**

We are going to use `info()` to check the missing data and data types for all columns.

In [ ]:
# use info() to check the missing data and datatype for all columns
test.info()
# the time series data has 10017 non-null entries with object datatype

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13512 entries, 0 to 13511
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   1990    10017 non-null  object
 1   1991    10017 non-null  object
 2   1992    10017 non-null  object
 3   1993    10017 non-null  object
 4   1994    10017 non-null  object
 5   1995    10017 non-null  object
 6   1996    10017 non-null  object
 7   1997    10017 non-null  object
 8   1998    10017 non-null  object
 9   1999    10017 non-null  object
 10  2000    10017 non-null  object
 11  2001    10017 non-null  object
 12  2002    10017 non-null  object
 13  2003    10017 non-null  object
 14  2004    10017 non-null  object
 15  2005    10017 non-null  object
 16  2006    10017 non-null  object
 17  2007    10017 non-null  object
 18  2008    10017 non-null  object
 19  2009    10017 non-null  object
 20  2010    10017 non-null  object
 21  2011    12382 non-null  object
dtypes: object(22)
memory u

Based on the information above, the time series data has 10017 non-null entries with object datatype. Since the data types for all columns are object, we will convert them to numeric for calculating the average.



### **Convert to numeric**

We are going to convert all columns to numeric and coerse the invalid pasring as Nan.

In [ ]:
# convert the entire dataframe from object to numeric
test = test.apply(pd.to_numeric, errors='coerce', axis=1) # use coerce to set invalid parsing as NaN 
test # show the time series data

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011
0,2.957481e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.957481e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.082346e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.082346e-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.967875e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.967875e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13507,2.497176e+06,2.693642e+06,2.909756e+06,3.139637e+06,3.373930e+06,3.605265e+06,3.817581e+06,4.024281e+06,4.229363e+06,4.439791e+06,4.661198e+06,4.898943e+06,5.148619e+06,5.410331e+06,5.683412e+06,5.967458e+06,6.275723e+06,6.597265e+06,6.932789e+06,7.283068e+06,7.648699e+06,NaN
13508,1.830400e+07,1.886488e+07,1.944609e+07,2.004848e+07,2.067294e+07,2.132040e+07,2.199214e+07,2.269759e+07,2.343868e+07,2.421743e+07,2.503600e+07,2.576921e+07,2.634556e+07,2.690436e+07,2.744822e+07,2.798869e+07,2.853356e+07,2.907984e+07,2.963688e+07,3.019380e+07,3.084463e+07,NaN
13509,1.012093e+07,1.056945e+07,1.106005e+07,1.156863e+07,1.206149e+07,1.251513e+07,1.299062e+07,1.342856e+07,1.385204e+07,1.429650e+07,1.478861e+07,1.542940e+07,1.612534e+07,1.686783e+07,1.764085e+07,1.843199e+07,1.933373e+07,2.026144e+07,2.121648e+07,2.220185e+07,2.321996e+07,NaN
13510,3.096861e+06,3.141668e+06,3.183257e+06,3.223515e+06,3.264940e+06,3.309118e+06,3.356932e+06,3.407476e+06,3.458431e+06,3.506648e+06,3.550144e+06,3.640719e+06,3.729883e+06,3.819641e+06,3.912871e+06,4.011828e+06,4.128987e+06,4.253139e+06,4.384859e+06,4.524564e+06,4.614728e+06,NaN


### **Compute the Average Across Columns**

After preprocessing the temporary dataframe,`years`, we are going to ignore NaNs and compute the row-wise average. The average values will be named as `Mean` and stored as a new column in df.

In [ ]:
# compute the average across all columns by using axis=1
# create a new column - Mean
df['Mean'] = test.mean(axis=1)
df.head() # show the first few rows of df after creating a new column

,Country code,Country name,Series code,Series name_x,SCALE,Decimals_x,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,Series name_y,Scale,Decimals_y,Order,Topic,Definition,Source,Mean
0,ABW,Aruba,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,29.5748,..,..,..,..,..,..,..,..,..,29.5748,..,..,..,..,..,..,..,..,..,..,..,Land area below 5m (% of land area),0,1,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,29.574810
1,ADO,Andorra,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..,Land area below 5m (% of land area),0,1,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,0.000000
2,AFG,Afghanistan,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0,..,..,..,..,..,..,..,..,..,0,..,..,..,..,..,..,..,..,..,..,..,Land area below 5m (% of land area),0,1,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,0.000000
3,AGO,Angola,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,0.208235,..,..,..,..,..,..,..,..,..,0.208235,..,..,..,..,..,..,..,..,..,..,..,Land area below 5m (% of land area),0,1,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,0.208235
4,ALB,Albania,AG.LND.EL5M.ZS,Land area below 5m (% of land area),0,1,4.96788,..,..,..,..,..,..,..,..,..,4.96788,..,..,..,..,..,..,..,..,..,..,..,Land area below 5m (% of land area),0,1,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,4.967875


## **Remove Repetitive Columns**

After merging both dataframes, there are some duplicated columns. We are going to remove them and rename the columns.

In [ ]:
# repetive columns -> drop list years axis=1, get rid of series name and decimals
# we drop the repetitive columns after the computation for the mean
# drop the time series data first
df = df.drop(list(year), axis=1)

# drop the duplicated columns in the dataframe
del df['Series name_y'] # drop the variable of 'Series name_y'
del df['Decimals_y'] # drop the variable of 'Decimals_y'
del df['SCALE'] # drop the variable of 'SCALE'

# rename some of the columns after drop the duplicated columns
df = df.rename(columns={'Series name_x': 'Series name',
                        'Decimals_x': 'Decimals'
                        })

# show the first few rows
df.head()

,Country code,Country name,Series code,Series name,Decimals,Scale,Order,Topic,Definition,Source,Mean
0,ABW,Aruba,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,29.574810
1,ADO,Andorra,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,0.000000
2,AFG,Afghanistan,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,0.000000
3,AGO,Angola,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,0.208235
4,ALB,Albania,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,4.967875


After removing `Series name_y`, `Decimals_y` and `SCALE`, we renamed `Series name_x` and `Decimals_x` to `Series name` and `Decimals`.

## **Drop Missing Values**

There are some NaNs and non-null values in the average columns. We will drop those rows in `df`.

In [ ]:
# check the NaNs and non-null values in the average column by using info()
df.info()
# there are 7891 non-null values in the average column and 13512 non-null values for the rest

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13512 entries, 0 to 13511
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country code  13512 non-null  object 
 1   Country name  13512 non-null  object 
 2   Series code   13512 non-null  object 
 3   Series name   13512 non-null  object 
 4   Decimals      13512 non-null  object 
 5   Scale         13512 non-null  object 
 6   Order         13512 non-null  int64  
 7   Topic         13512 non-null  object 
 8   Definition    13512 non-null  object 
 9   Source        13512 non-null  object 
 10  Mean          7891 non-null   float64
dtypes: float64(1), int64(1), object(9)
memory usage: 1.2+ MB


In [ ]:
# drop the missing values
df.dropna(inplace=True)

In [ ]:
# check the shape and info of our dataframe after the data cleaning
print(df.shape) # our cleaned dataframe contains 7891 rows and 11 columns
df.info() # there are 7891 non-null values

(7891, 11)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7891 entries, 0 to 13511
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country code  7891 non-null   object 
 1   Country name  7891 non-null   object 
 2   Series code   7891 non-null   object 
 3   Series name   7891 non-null   object 
 4   Decimals      7891 non-null   object 
 5   Scale         7891 non-null   object 
 6   Order         7891 non-null   int64  
 7   Topic         7891 non-null   object 
 8   Definition    7891 non-null   object 
 9   Source        7891 non-null   object 
 10  Mean          7891 non-null   float64
dtypes: float64(1), int64(1), object(9)
memory usage: 739.8+ KB


After removing all missing values, we will have 7891 rows and 11 columns in our cleaned dataframe.

# **Filter `df` by Group's List of Countries**

We are going to filter our group list of countries from df`. It includes South Korea, Israel, Italy, Ireland and Iceland.

In [ ]:
# filter df by group list of countries
# we are team 5: our group list of countries is 'IRL','ISL','ITA','ISR','KAZ','KOR'
# use a list of values to select rows from our dataframe  
my_countries = ['IRL','ISL','ITA','ISR','KAZ','KOR']
df = df[df["Country code"].isin(my_countries)]
df # show the dateframe

,Country code,Country name,Series code,Series name,Decimals,Scale,Order,Topic,Definition,Source,Mean
94,IRL,Ireland,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,3.958941e+00
97,ISL,Iceland,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,4.412560e+00
98,ISR,Israel,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,7.823743e+00
99,ITA,Italy,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,5.188644e+00
103,KAZ,Kazakhstan,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,6.678062e+00
...,...,...,...,...,...,...,...,...,...,...,...
13376,ISL,Iceland,SP.URB.TOTL,Urban population,0,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...,2.610888e+05
13377,ISR,Israel,SP.URB.TOTL,Urban population,0,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...,5.691833e+06
13378,ITA,Italy,SP.URB.TOTL,Urban population,0,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...,3.888347e+07
13382,KAZ,Kazakhstan,SP.URB.TOTL,Urban population,0,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...,8.824369e+06


In [ ]:
# check our work again
df['Country code'].value_counts()
# there are 38 rows for Kazakhstan, 36 rows for South Korea, 36 rows for Israel, 34 rows for Italy, 32 rows for Ireland and 30 rows for Iceland
# it finally comes up with 206 rows

KAZ    38
KOR    36
ISR    36
ITA    34
IRL    32
ISL    30
Name: Country code, dtype: int64

We made a variable, `my_countries`, and used `df.isin()` to subset the countries in our group list. By using `df.value_counts()`, there are 38 rows for Kazakhstan, 36 rows for South Korea, 36 rows for Israel, 34 rows for Italy, 32 rows for Ireland and 30 rows for Iceland. It finally comes up with 206 rows in the dataframe.

# **Eliminate Series with Less Than 6 Occurrence**

To identify the set of series for which we have data for all countries, we need a common set of variables for all countries. We can see that variables appear different time in each series. We will pick the variables with 6 times occurrence as they are common to all countries.

In [ ]:
# group the Series name together
tmp = df.groupby('Series name')

In [ ]:
# find the occurence for each variables by using count
tmp.count()

,Country code,Country name,Series code,Decimals,Scale,Order,Topic,Definition,Source,Mean
Series name,,,,,,,,,,
Access to electricity (% of total population),1,1,1,1,1,1,1,1,1,1
Access to improved sanitation (% of total pop.),5,5,5,5,5,5,5,5,5,5
Access to improved water source (% of total pop.),6,6,6,6,6,6,6,6,6,6
Agricultural land under irrigation (% of total ag. land),3,3,3,3,3,3,3,3,3,3
Annual freshwater withdrawals (% of internal resources),5,5,5,5,5,5,5,5,5,5
"Average annual precipitation (1961-1990, mm)",6,6,6,6,6,6,6,6,6,6
CO2 emissions per capita (metric tons),6,6,6,6,6,6,6,6,6,6
"CO2 emissions per units of GDP (kg/$1,000 of 2005 PPP $)",6,6,6,6,6,6,6,6,6,6
"CO2 emissions, total (KtCO2)",6,6,6,6,6,6,6,6,6,6


In [ ]:
# save tmp.count as tmp2
tmp2 = tmp.count()

In [ ]:
# check the info of tmp2
# we have 44 series in our dataframe
tmp2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, Access to electricity (% of total population) to Urban population growth (annual %)
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Country code  44 non-null     int64
 1   Country name  44 non-null     int64
 2   Series code   44 non-null     int64
 3   Decimals      44 non-null     int64
 4   Scale         44 non-null     int64
 5   Order         44 non-null     int64
 6   Topic         44 non-null     int64
 7   Definition    44 non-null     int64
 8   Source        44 non-null     int64
 9   Mean          44 non-null     int64
dtypes: int64(10)
memory usage: 3.8+ KB


In [ ]:
# show the dataframe of tmp2
tmp2

,Country code,Country name,Series code,Decimals,Scale,Order,Topic,Definition,Source,Mean
Series name,,,,,,,,,,
Access to electricity (% of total population),1,1,1,1,1,1,1,1,1,1
Access to improved sanitation (% of total pop.),5,5,5,5,5,5,5,5,5,5
Access to improved water source (% of total pop.),6,6,6,6,6,6,6,6,6,6
Agricultural land under irrigation (% of total ag. land),3,3,3,3,3,3,3,3,3,3
Annual freshwater withdrawals (% of internal resources),5,5,5,5,5,5,5,5,5,5
"Average annual precipitation (1961-1990, mm)",6,6,6,6,6,6,6,6,6,6
CO2 emissions per capita (metric tons),6,6,6,6,6,6,6,6,6,6
"CO2 emissions per units of GDP (kg/$1,000 of 2005 PPP $)",6,6,6,6,6,6,6,6,6,6
"CO2 emissions, total (KtCO2)",6,6,6,6,6,6,6,6,6,6


In [ ]:
# since we have 6 countries
# filter the series with more than occurrence in our dataframe
MoreThan6 =  tmp2[(tmp2['Country code'] >= 6) & (tmp2['Country name'] >= 6) & (tmp2['Series code'] >= 6) & (tmp2['Scale'] >= 6) & (tmp2['Decimals'] >= 6) & 
                  (tmp2['Order'] >= 6) & (tmp2['Topic'] >= 6) & (tmp2['Definition'] >= 6) & (tmp2['Source'] >= 6) & (tmp2['Mean'] >= 6)]

In [ ]:
# show the series with at least 6 occurrence
MoreThan6

,Country code,Country name,Series code,Decimals,Scale,Order,Topic,Definition,Source,Mean
Series name,,,,,,,,,,
Access to improved water source (% of total pop.),6,6,6,6,6,6,6,6,6,6
"Average annual precipitation (1961-1990, mm)",6,6,6,6,6,6,6,6,6,6
CO2 emissions per capita (metric tons),6,6,6,6,6,6,6,6,6,6
"CO2 emissions per units of GDP (kg/$1,000 of 2005 PPP $)",6,6,6,6,6,6,6,6,6,6
"CO2 emissions, total (KtCO2)",6,6,6,6,6,6,6,6,6,6
Ease of doing business (ranking 1-183; 1=best),6,6,6,6,6,6,6,6,6,6
Energy use per capita (kilograms of oil equivalent),6,6,6,6,6,6,6,6,6,6
"Energy use per units of GDP (kg oil eq./$1,000 of 2005 PPP $)",6,6,6,6,6,6,6,6,6,6
"Foreign direct investment, net inflows (% of GDP)",6,6,6,6,6,6,6,6,6,6


In [ ]:
# check the info of filtered series
# we have 27 series in our dataframe
MoreThan6.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27 entries, Access to improved water source (% of total pop.) to Urban population growth (annual %)
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Country code  27 non-null     int64
 1   Country name  27 non-null     int64
 2   Series code   27 non-null     int64
 3   Decimals      27 non-null     int64
 4   Scale         27 non-null     int64
 5   Order         27 non-null     int64
 6   Topic         27 non-null     int64
 7   Definition    27 non-null     int64
 8   Source        27 non-null     int64
 9   Mean          27 non-null     int64
dtypes: int64(10)
memory usage: 2.3+ KB


## **Create `shared_list`**

In [ ]:
# create a new variable, shared_list, to share the series name with at least 6 occurences
shared_list = MoreThan6.index
shared_list # show the list
# there are 27 series in the list

Index(['Access to improved water source (% of total pop.)',
       'Average annual precipitation (1961-1990, mm)',
       'CO2 emissions per capita (metric tons)',
       'CO2 emissions per units of GDP (kg/$1,000 of 2005 PPP $)',
       'CO2 emissions, total (KtCO2)',
       'Ease of doing business (ranking 1-183; 1=best)',
       'Energy use per capita (kilograms of oil equivalent)',
       'Energy use per units of GDP (kg oil eq./$1,000 of 2005 PPP $)',
       'Foreign direct investment, net inflows (% of GDP)', 'GDP ($)',
       'GNI per capita (Atlas $)', 'Land area below 5m (% of land area)',
       'Methane (CH4) emissions, total (KtCO2e)',
       'Nationally terrestrial protected areas (% of total land area)',
       'Nitrous oxide (N2O) emissions, total (KtCO2e)',
       'Nurses and midwives (per 1,000 people)',
       'Other GHG emissions, total (KtCO2e)', 'Paved roads (% of total roads)',
       'Physicians (per 1,000 people)', 'Population',
       'Population below 5m (% of

In [ ]:
# subset the rows from df that Series name is in the shared list
df = df[df["Series name"].isin(shared_list)]
df.info() # check the info and find that we have 162 rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162 entries, 94 to 13388
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country code  162 non-null    object 
 1   Country name  162 non-null    object 
 2   Series code   162 non-null    object 
 3   Series name   162 non-null    object 
 4   Decimals      162 non-null    object 
 5   Scale         162 non-null    object 
 6   Order         162 non-null    int64  
 7   Topic         162 non-null    object 
 8   Definition    162 non-null    object 
 9   Source        162 non-null    object 
 10  Mean          162 non-null    float64
dtypes: float64(1), int64(1), object(9)
memory usage: 15.2+ KB


In [ ]:
# check how many topics are available in our dataframe
df['Topic'].unique()
# there are 5 topics available - Exposure to impacts, Resilience, GHG emissions and energy use, Climate, Size of the economy

array(['Exposure to impacts', 'Resilience',
       'GHG emissions and energy use', 'Climate', 'Size of the economy'],
      dtype=object)

After filtering the series, 27 series left in our dataframe that has at least 6 occurence. Then we created a new variable,`shared_list`, to subset the related rows in `df`. `df` finally has 162 rows with non-null values. There are 5 topics available in `df` that include Exposure to impacts, Resilience, GHG emissions and energy use, Climate, Size of the economy.

# **Extract Parameters for the Optimization Code**

We will extract parameters by country and series and exclude the series of GHG emissions and energy use.

In [ ]:
# subset rows without topic of GHG emissions and energy use
tmp = df # make a copy of df
tmp[tmp['Topic'] != 'GHG emissions and energy use'] # subset the rows where the topic is not equal to GHG emissions and energy use

,Country code,Country name,Series code,Series name,Decimals,Scale,Order,Topic,Definition,Source,Mean
94,IRL,Ireland,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,3.958941e+00
97,ISL,Iceland,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,4.412560e+00
98,ISR,Israel,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,7.823743e+00
99,ITA,Italy,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,5.188644e+00
103,KAZ,Kazakhstan,AG.LND.EL5M.ZS,Land area below 5m (% of land area),1,0,11,Exposure to impacts,Land area below 5m is the percentage of total ...,Center for International Earth Science Informa...,6.678062e+00
...,...,...,...,...,...,...,...,...,...,...,...
13376,ISL,Iceland,SP.URB.TOTL,Urban population,0,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...,2.610888e+05
13377,ISR,Israel,SP.URB.TOTL,Urban population,0,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...,5.691833e+06
13378,ITA,Italy,SP.URB.TOTL,Urban population,0,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...,3.888347e+07
13382,KAZ,Kazakhstan,SP.URB.TOTL,Urban population,0,0,14,Exposure to impacts,Urban population is the share of the midyear p...,World Bank staff estimates based on United Nat...,8.824369e+06


We made a copy of `df` as a temporary dataframe and subset the rows without the topic of GHG emissions and energy use. The temporary dataframe have 114 rows now.

# **Create a Dictionary**

In this section, we are going to use `Country code`, `Series code` and `Mean` to create a dictionary for modeling.

## **Make a dictionary with key and value**

We will use `zip()` two keys and value together and make a tuple. Then, we will convert it to dictionary.

In [ ]:
# return an iterator and generate a series of tuples
x1 = zip(tmp["Country code"], tmp['Series code']) # use zip to match Country code and Series code together

In [ ]:
x1 = zip(zip(tmp["Country code"], tmp['Series code']), tmp['Mean']) # use zip to match them together

In [ ]:
# convert this to a dictionary based on the previous codes
x1 = dict(zip(zip(tmp["Country code"], tmp['Series code']), tmp['Mean']))

In [ ]:
# apply it to our dataset and name it var_dict
var_dict = dict(zip(zip(tmp["Country code"], tmp['Series code']), tmp['Mean']))
print(len(var_dict)) # check the length of var_dict

162


After the steps, we tried to apply on our dataset and named it as `var_dict`.

## **Make Three New Variables**

We will make three new variables that have unique values which includes `countries`, `indicators`, and `mygroups`. All of these will be used in our model later.

In [ ]:
# make three new variables that have unique values

# get the unique values from Country code as countries
countries = df['Country code'].unique() # we have 6 countries in our dataframe - Kazakhstan, South Korea, Israel, Italy, Ireland and Iceland

# get the unique values from Series code as indicators
indicators = df['Series code'].unique() # we have 27 indicators in our dataframe

# get the unique values from Topic as mygroups
mygroups = df['Topic'].unique() # we have 5 topics in our dataframe - Exposure to impacts, Resilience, GHG emissions and energy use, Climate, Size of the economy

In [ ]:
# make all new variables as list 
countries = list(countries)
print(type(countries))
indicators = list(indicators)
print(type(indicators))
mygroups = list(mygroups)
print(type(mygroups))

<class 'list'>
<class 'list'>
<class 'list'>


In [ ]:
# remove GHG emissions and energy use
mygroups.remove('GHG emissions and energy use')

Finally, we has 6 countries in our dataframe, including Kazakhstan, South Korea, Israel, Italy, Ireland and Iceland. We also have 27 indicators and 5 topics in our dataframe. Since we will not use variables from GHG emissions and energy use to predict dioxide emission `EM.ATM.CO2E.KT`. We will only use variables of 'Exposure to impacts', 'Resilience' and 'Size of the economy' to predict the greenhouse gas variable `EN.ATM.CO2E.KT`.

## **Extract list of series per Topic**



We are going to make a new variable, `Topics`, that contains all topic groups without GHG emissions and energy use. Then, we will create a new variable, `indicator_dict`, as an empty dictionary that will be used to force our model to take one variable from each group.

In [ ]:
# create a variable, topics, for all of the topic groups without GHG emissions and energy use
top = df[df['Topic'] != 'GHG emissions and energy use']
topics = list((top['Topic']).unique()) 
topics
#remove GHG emissions and energy use

['Exposure to impacts', 'Resilience', 'Climate', 'Size of the economy']

In [ ]:
# after eliminate the topic of GHG emissions and energy use
# create the indicator dictionary 
indicator_dict = dict()
for topic in topics:
  codes = df[df['Topic'] == topic]['Series code'].unique()
  indicator_dict[topic] = list(codes)
indicator_dict

{'Climate': ['EN.CLC.HPPT.MM'],
 'Exposure to impacts': ['AG.LND.EL5M.ZS',
  'EN.POP.EL5M.ZS',
  'ER.LND.PTLD.ZS',
  'SH.DYN.MORT',
  'SP.URB.GROW',
  'SP.URB.TOTL'],
 'Resilience': ['BX.KLT.DINV.WD.GD.ZS',
  'IC.BUS.EASE.XQ',
  'IS.ROD.PAVE.ZS',
  'SE.ENR.PRSC.FM.ZS',
  'SE.PRM.CMPT.ZS',
  'SH.H2O.SAFE.ZS',
  'SH.MED.NUMW.P3',
  'SH.MED.PHYS.ZS'],
 'Size of the economy': ['NY.GDP.MKTP.CD',
  'NY.GNP.PCAP.CD',
  'SP.POP.GROW',
  'SP.POP.TOTL']}

In [ ]:
# indicators without GHG emissions and energy in the series code
indicators = top['Series code'].unique()
len(indicators)
# finally indicators have 19 indicators in our dataframe

19

In [ ]:
# set co2 - `EM.ATM.CO2E.KT`
co2 = df[df['Series code'] == "EN.ATM.CO2E.KT"]

# generate the mean for each country in our group list
y = {}

for c in countries:
  y[c] = co2[co2['Country code'] == c]['Mean'].item()

# show the result
y

{'IRL': 38415.10600000001,
 'ISL': 2118.754,
 'ISR': 53577.379,
 'ITA': 443602.78,
 'KAZ': 173398.19341176475,
 'KOR': 401974.22399999993}

By creating the dictionary, it will be used to force our model to take one variable from each group. The mean values of CO2 for Ireland, Iceland, Israel, Italy, Kazakhstan and South Korea are 38415, 2118, 53577, 443602, 173398 and 401974 respectively.

# **Model**
---
In this section we will be building a model having 6 rows of data pertaining to one country each and 19 predictor variables. 

As per the instructions given only 4 predictors are used for the model. 


## **Concrete Model**

In [ ]:
# declare the model
model = ConcreteModel()

## **Decision Variables**
---
Below are the details of the decision variables:

* `a` is the coefficients on the terms in our model - one `a` for each variable using `Var(indicators, domain=XYZ, bounds=ABC)`.
* `activation` is whether or not the variable turns on (1) or not (0). Hence, one `activation` for each variable in the model using `Var(indicators, domain=XYZ, bounds=ABC)`.
* `y` is a decision variable which enforces the linear form of the model (these are our intermediate predictions for each country, so we should only have 6) using `Var(countries, domain=XYZ, bounds=ABC, initialize=Z)`.

In [ ]:
# declare decision variables
model.a = Var(indicators, domain=Reals,bounds = (-100,100)) # coefficients on the terms in our model
model.activation = Var(indicators, domain=Binary) # whether or not the variable turns on (1) or not (0)
model.y = Var(countries, domain= NonNegativeReals) # variable which enforces the linear form of the mode

## **Constraints**
---
In this section we declare all the prediction values, constraints as per the problem statement.

In [ ]:
# Constraints
model.predictions = ConstraintList() 

# Predicted value
# model.a[i] indicates the coefficient of variables
# the values from var_dict indicates values of variables 
for c in countries:
  pred = 0
  for i in indicators:
    pred += (model.a[i]*var_dict[c, i])
  model.predictions.add(model.y[c] == pred)

model.constraints = ConstraintList() 

# use data from up to 4 different features and at least one feature per category
for g in mygroups:
  ac = 0
  for i in indicator_dict[g]:
    ac += model.activation[i] # it will be positive
  model.constraints.add(ac >= 1) # take at least one series

ac2 = 0
for i in indicators:
  ac2 += model.activation[i] # it will be positive
model.constraints.add(ac2 <= 4) # take 4 different series at most

model.activations = ConstraintList() 

# Conditional Constraints 
for i in indicators:
# since the value of each parameter should be between -100 and 100
# we set the bounds for its associated value in the linear regression model will be 0 if the model does not use a specific parameter
# it forces the activation variable to be 1 if the series is between -100 and 100
# it forces the activation variable to be 0 if the series is 0
  model.activations.add(model.a[i] <= 100*model.activation[i])
  model.activations.add(model.a[i] >= -100*model.activation[i])


# declare objective
obj_expr = 0
for c in countries:
  obj_expr += ((model.y[c] - y[c])/1000)**2 # divide each error term by 1000 before squaring it to generate a smaller objective value   
model.error = Objective(
                      expr = obj_expr,
                      sense = minimize) # to get the least square error, we use minimize


# show the model we've created
model.pprint()

6 Set Declarations
    a_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   19 : {'AG.LND.EL5M.ZS', 'BX.KLT.DINV.WD.GD.ZS', 'EN.CLC.HPPT.MM', 'EN.POP.EL5M.ZS', 'ER.LND.PTLD.ZS', 'IC.BUS.EASE.XQ', 'IS.ROD.PAVE.ZS', 'NY.GDP.MKTP.CD', 'NY.GNP.PCAP.CD', 'SE.ENR.PRSC.FM.ZS', 'SE.PRM.CMPT.ZS', 'SH.DYN.MORT', 'SH.H2O.SAFE.ZS', 'SH.MED.NUMW.P3', 'SH.MED.PHYS.ZS', 'SP.POP.GROW', 'SP.POP.TOTL', 'SP.URB.GROW', 'SP.URB.TOTL'}
    activation_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   19 : {'AG.LND.EL5M.ZS', 'BX.KLT.DINV.WD.GD.ZS', 'EN.CLC.HPPT.MM', 'EN.POP.EL5M.ZS', 'ER.LND.PTLD.ZS', 'IC.BUS.EASE.XQ', 'IS.ROD.PAVE.ZS', 'NY.GDP.MKTP.CD', 'NY.GNP.PCAP.CD', 'SE.ENR.PRSC.FM.ZS', 'SE.PRM.CMPT.ZS', 'SH.DYN.MORT', 'SH.H2O.SAFE.ZS', 'SH.MED.NUMW.P3', 'SH.MED.PHYS.ZS', 'SP.POP.GROW', 'SP.POP.TOTL', 'SP.URB.GROW', 'SP.URB.TOTL'}
    activations_index : Size=1

## **Solver Results**

In [ ]:
# solve it
SolverFactory('bonmin', executable='/content/bonmin').solve(model).write()

# show the results
print("Objective value = ", model.error())

for g in mygroups:
  for i in indicator_dict[g]:
    print(g, i, model.a[i](), model.activation[i]())

for c in countries:
  print(c,': Predicted Value=', model.y[c](),', Actual Value=',y[c])


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 44
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: bonmin\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 8.93872618675232
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
Objective value =  57.486184477

## **Comparision of results to actual**

The Objective Function value which shows the error between Predicted and Actual values is found to be `57.48`

Objective value =  `57.48618447731851`

Below are the details of the Indicators and the optimal series which are having an activation value of 1.

* Exposure to impacts - `SP.URB.TOTL` 

* Resilience - `SE.ENR.PRSC.FM.ZS`

* Climate - `EN.CLC.HPPT.MM`

* Size of the economy - `SP.POP.TOTL`


Below are the details of predicted and actual values country wise-

* IRL : Predicted Value= `33586.62189830973` , Actual Value= `38415.10600000001`
* ISL : Predicted Value= `7836.6810655847385` , Actual Value= `2118.754`
* ISR : Predicted Value= `52486.48411998876` , Actual Value= `53577.379`
* ITA : Predicted Value= `444118.6611967002` , Actual Value= `443602.78`
* KAZ : Predicted Value= `173280.507599581` , Actual Value= `173398.19341176475`
* KOR : Predicted Value= `401889.36628990027` , Actual Value= `401974.22399999993`

We can observe that there is less error between predicted and actual values for almost all the countries excelt ISL. For ISL, we see that there is a high deviation from the actual value. To further analyse, let us check on how the results change with the increase or decrease in prediction variables.


## **Experiments**
---
In this section we are going to run the Model again for various predictor variables like 5,3,2 and then compare how the results like Objective function and predictors values are varying.

### **With 5 Predictors variables and having atleast one in each group**

In [ ]:
# declare the model
model2 = ConcreteModel()

# declare decision variables
model2.a = Var(indicators, domain=Reals,bounds = (-100,100))
model2.activation = Var(indicators, domain=Binary)
model2.y = Var(countries, domain= NonNegativeReals) 

# Constraints
model2.predictions = ConstraintList()

# Predicted value
# model.a[i] indicates the coefficient of variables
# the values from var_dict indicates values of variables
for c in countries:
  pred = 0
  for i in indicators:
    pred += (model2.a[i]*var_dict[c, i])
  model2.predictions.add(model2.y[c] == pred)

model2.constraints = ConstraintList() 

for g in mygroups:
  ac = 0
  for i in indicator_dict[g]:
    ac += model2.activation[i] # it will be positive
  model2.constraints.add(ac >= 1) # take at least one series

ac2 = 0
for i in indicators:
  ac2 += model2.activation[i] # it will be positive
model2.constraints.add(ac2 <= 5) # take 5 different series at most

# Conditional Constraints 

model2.activations = ConstraintList() 

for i in indicators:
# since the value of each parameter should be between -100 and 100
# we set the bounds for its associated value in the linear regression model will be 0 if the model does not use a specific parameter
# it forces the activation variable to be 1 if the series is between -100 and 100
# it forces the activation variable to be 0 if the series is 0
  model2.activations.add(model2.a[i] <= 100*model2.activation[i])
  model2.activations.add(model2.a[i] >= -100*model2.activation[i])   


# declare objective
obj_expr = 0
for c in countries:
  obj_expr += ((model2.y[c] - y[c])/1000)**2 # divide each error term by 1000 before squaring it to generate a smaller objective value   
model2.error = Objective(
                      expr = obj_expr,
                      sense = minimize) # to get the least square error, we use minimize


# show the model we've created
model2.pprint()


6 Set Declarations
    a_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   19 : {'AG.LND.EL5M.ZS', 'BX.KLT.DINV.WD.GD.ZS', 'EN.CLC.HPPT.MM', 'EN.POP.EL5M.ZS', 'ER.LND.PTLD.ZS', 'IC.BUS.EASE.XQ', 'IS.ROD.PAVE.ZS', 'NY.GDP.MKTP.CD', 'NY.GNP.PCAP.CD', 'SE.ENR.PRSC.FM.ZS', 'SE.PRM.CMPT.ZS', 'SH.DYN.MORT', 'SH.H2O.SAFE.ZS', 'SH.MED.NUMW.P3', 'SH.MED.PHYS.ZS', 'SP.POP.GROW', 'SP.POP.TOTL', 'SP.URB.GROW', 'SP.URB.TOTL'}
    activation_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   19 : {'AG.LND.EL5M.ZS', 'BX.KLT.DINV.WD.GD.ZS', 'EN.CLC.HPPT.MM', 'EN.POP.EL5M.ZS', 'ER.LND.PTLD.ZS', 'IC.BUS.EASE.XQ', 'IS.ROD.PAVE.ZS', 'NY.GDP.MKTP.CD', 'NY.GNP.PCAP.CD', 'SE.ENR.PRSC.FM.ZS', 'SE.PRM.CMPT.ZS', 'SH.DYN.MORT', 'SH.H2O.SAFE.ZS', 'SH.MED.NUMW.P3', 'SH.MED.PHYS.ZS', 'SP.POP.GROW', 'SP.POP.TOTL', 'SP.URB.GROW', 'SP.URB.TOTL'}
    activations_index : Size=1

In [ ]:
# solve it
SolverFactory('bonmin', executable='/content/bonmin').solve(model2).write()

# show the results
print("Objective value = ", model2.error())

for g in mygroups:
  for i in indicator_dict[g]:
    print(g, i, model2.a[i](), model2.activation[i]())

for c in countries:
  print(c,': Predicted Value=', model2.y[c](),', Actual Value=',y[c])

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 44
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: bonmin\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 1.6656203269958496
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
Objective value =  9.43786202

Below are the results obtained-

Objective value =  `9.43786202625372`

The error obtained is very less compared to what we observed for 4 predictor variables.

* IRL : Predicted Value= `36450.971373830376` , Actual Value= `38415.10600000001`
* ISL : Predicted Value= `4110.970622087293` , Actual Value= `2118.754`
* ISR : Predicted Value= `54737.2301057426` , Actual Value= `53577.379`
* ITA : Predicted Value= `443805.5720556702` , Actual Value= `443602.78`
* KAZ : Predicted Value= `173708.4004702595` , Actual Value= `173398.19341176475`
* KOR : Predicted Value= `401615.75154256553` , Actual Value= `401974.22399999993`

We see that the error between the Predicted and Actual values are very less compared to what we observed for 4 predictions. The same is evident for ISL as well where the error reduced there by giving a less possible value to our Objective function which denoted the sum of squared errors. 

### **With 3 Predictor Variables**
---
We tried reducing number of parameters from 4 to 3 to see how the result varies and noticed it provided an infeasible solution as all 4 groups cannot have atleast 1 series code selected as the atmost value is 3. 

We tried to build the model without the constraint but it shows that one group has two variables and it is not our assumption of trying to have at least one variable for each group. Since we will have only three variables, it's not possible for every group to have at least one variable. Let's force three of them to have one variable.

In [ ]:
# declare the model
model3 = ConcreteModel()

# declare decision variables
model3.a = Var(indicators, domain=Reals,bounds = (-100,100))
model3.activation = Var(indicators, domain=Binary)
model3.y = Var(countries, domain= NonNegativeReals) 

# Constraints
model3.predictions = ConstraintList() 

# Predicted value
# model.a[i] indicates the coefficient of variables
# the values from var_dict indicates values of variables
for c in countries:
  pred = 0
  for i in indicators:
    pred += (model3.a[i]*var_dict[c, i])
  model3.predictions.add(model3.y[c] == pred)

model3.constraints = ConstraintList() 

for g in mygroups:
  ac = 0
  for i in indicator_dict[g]:
    ac += model3.activation[i] # it will be positive
  model3.constraints.add(ac <= 1) # force it not to take more than one

ac2 = 0
for i in indicators:
  ac2 += model3.activation[i] # it will be positive
model3.constraints.add(ac2 == 3) # take exact 3 different series

model3.activations = ConstraintList() 
# Conditional Constraints 

for i in indicators:
# since the value of each parameter should be between -100 and 100
# we set the bounds for its associated value in the linear regression model will be 0 if the model does not use a specific parameter
# it forces the activation variable to be 1 if the series is between -100 and 100
# it forces the activation variable to be 0 if the series is 0
  model3.activations.add(model3.a[i] <= 100*model3.activation[i])
  model3.activations.add(model3.a[i] >= -100*model3.activation[i])   


# declare objective
obj_expr = 0
for c in countries:
  obj_expr += ((model3.y[c] - y[c])/1000)**2 # divide each error term by 1000 before squaring it to generate a smaller objective value   
model3.error = Objective(
                      expr = obj_expr,
                      sense = minimize) # to get the least square error, we use minimize


# show the model we've created
model3.pprint()


6 Set Declarations
    a_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   19 : {'AG.LND.EL5M.ZS', 'BX.KLT.DINV.WD.GD.ZS', 'EN.CLC.HPPT.MM', 'EN.POP.EL5M.ZS', 'ER.LND.PTLD.ZS', 'IC.BUS.EASE.XQ', 'IS.ROD.PAVE.ZS', 'NY.GDP.MKTP.CD', 'NY.GNP.PCAP.CD', 'SE.ENR.PRSC.FM.ZS', 'SE.PRM.CMPT.ZS', 'SH.DYN.MORT', 'SH.H2O.SAFE.ZS', 'SH.MED.NUMW.P3', 'SH.MED.PHYS.ZS', 'SP.POP.GROW', 'SP.POP.TOTL', 'SP.URB.GROW', 'SP.URB.TOTL'}
    activation_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   19 : {'AG.LND.EL5M.ZS', 'BX.KLT.DINV.WD.GD.ZS', 'EN.CLC.HPPT.MM', 'EN.POP.EL5M.ZS', 'ER.LND.PTLD.ZS', 'IC.BUS.EASE.XQ', 'IS.ROD.PAVE.ZS', 'NY.GDP.MKTP.CD', 'NY.GNP.PCAP.CD', 'SE.ENR.PRSC.FM.ZS', 'SE.PRM.CMPT.ZS', 'SH.DYN.MORT', 'SH.H2O.SAFE.ZS', 'SH.MED.NUMW.P3', 'SH.MED.PHYS.ZS', 'SP.POP.GROW', 'SP.POP.TOTL', 'SP.URB.GROW', 'SP.URB.TOTL'}
    activations_index : Size=1

In [ ]:
# solve it
SolverFactory('bonmin', executable='/content/bonmin').solve(model3).write()

# show the results
print("Objective value = ", model3.error())

for g in mygroups:
  for i in indicator_dict[g]:
    print(g, i, model3.a[i](), model3.activation[i]())

for c in countries:
  print(c,': Predicted Value=', model3.y[c](),', Actual Value=',y[c])

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 44
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: bonmin\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 3.112157106399536
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
Objective value =  9.475278776

We see that the objective value is 9.475 and below are the details of the groups that are included in the optimal solution- 


* Exposure to impacts -` SP.URB.TOTL`

* Resilience - `IS.ROD.PAVE.ZS`

* Size of the economy - `SP.POP.TOTL`

Below are the Predicted and actuals values-

* IRL : Predicted Value= `36524.061693995085` , Actual Value= `38415.10600000001`
* ISL : Predicted Value= `4252.453771936094` , Actual Value= `2118.754`
* ISR : Predicted Value= `54632.65160875411` , Actual Value= `53577.379`
* ITA : Predicted Value= `443797.3456131321` , Actual Value=` 443602.78`
* KAZ : Predicted Value= `173687.1049982201 `, Actual Value= `173398.19341176475`
* KOR : Predicted Value= `401640.1139488336` , Actual Value= `401974.22399999993`



### **Running model by removing condition of one in each group and maximum of 4 predictors**

In [ ]:
# declare the model
model4 = ConcreteModel()

# declare decision variables
model4.a = Var(indicators, domain=Reals,bounds = (-100,100))
model4.activation = Var(indicators, domain=Binary)
model4.y = Var(countries, domain= NonNegativeReals) 

# Constraints
model4.predictions = ConstraintList() 

# Predicted value
# model.a[i] indicates the coefficient of variables
# the values from var_dict indicates values of variables
for c in countries:
  pred = 0
  for i in indicators:
    pred += (model4.a[i]*var_dict[c, i])
  model4.predictions.add(model4.y[c] == pred)

model4.constraints = ConstraintList() 

ac2 = 0
for i in indicators:
  ac2 += model4.activation[i] # it will be positive
model4.constraints.add(ac2 <= 4)  # take 4 different series at most

model4.activations = ConstraintList() 
# Conditional Constraints 

for i in indicators:
# since the value of each parameter should be between -100 and 100
# we set the bounds for its associated value in the linear regression model will be 0 if the model does not use a specific parameter
# it forces the activation variable to be 1 if the series is between -100 and 100
# it forces the activation variable to be 0 if the series is 0
  model4.activations.add(model4.a[i] <= 100*model4.activation[i])
  model4.activations.add(model4.a[i] >= -100*model4.activation[i])   


# declare objective
obj_expr = 0
for c in countries:
  obj_expr += ((model4.y[c] - y[c])/1000)**2 # divide each error term by 1000 before squaring it to generate a smaller objective value   
model4.error = Objective(
                      expr = obj_expr,
                      sense = minimize) # to get the least square error, we use minimize


# show the model we've created
model4.pprint()

6 Set Declarations
    a_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   19 : {'AG.LND.EL5M.ZS', 'BX.KLT.DINV.WD.GD.ZS', 'EN.CLC.HPPT.MM', 'EN.POP.EL5M.ZS', 'ER.LND.PTLD.ZS', 'IC.BUS.EASE.XQ', 'IS.ROD.PAVE.ZS', 'NY.GDP.MKTP.CD', 'NY.GNP.PCAP.CD', 'SE.ENR.PRSC.FM.ZS', 'SE.PRM.CMPT.ZS', 'SH.DYN.MORT', 'SH.H2O.SAFE.ZS', 'SH.MED.NUMW.P3', 'SH.MED.PHYS.ZS', 'SP.POP.GROW', 'SP.POP.TOTL', 'SP.URB.GROW', 'SP.URB.TOTL'}
    activation_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   19 : {'AG.LND.EL5M.ZS', 'BX.KLT.DINV.WD.GD.ZS', 'EN.CLC.HPPT.MM', 'EN.POP.EL5M.ZS', 'ER.LND.PTLD.ZS', 'IC.BUS.EASE.XQ', 'IS.ROD.PAVE.ZS', 'NY.GDP.MKTP.CD', 'NY.GNP.PCAP.CD', 'SE.ENR.PRSC.FM.ZS', 'SE.PRM.CMPT.ZS', 'SH.DYN.MORT', 'SH.H2O.SAFE.ZS', 'SH.MED.NUMW.P3', 'SH.MED.PHYS.ZS', 'SP.POP.GROW', 'SP.POP.TOTL', 'SP.URB.GROW', 'SP.URB.TOTL'}
    activations_index : Size=1

In [ ]:
# solve it
SolverFactory('bonmin', executable='/content/bonmin').solve(model4).write()

# show the results
print("Objective value = ", model4.error())

for g in mygroups:
  for i in indicator_dict[g]:
    print(g, i, model4.a[i](), model4.activation[i]())

for c in countries:
  print(c,': Predicted Value=', model4.y[c](),', Actual Value=',y[c])

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 44
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: bonmin\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 1.7759435176849365
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
Objective value =  9.47527876

In this case we are removing the condition of at least one group each and taking a maximum of 4 predictions. We see that the sokution provided is optimum  and the sum of errors is-

Objective value =  `9.475278762910749`

Below are the optimal group values for this type of model-

* Exposure to impacts - `SP.URB.TOTL` 

* Resilience - `IS.ROD.PAVE.ZS`

* Size of the economy - `NY.GDP.MKTP.CD`

* Size of the economy - `SP.POP.TOTL`

Below are the details of the actual and Predicted values-

* IRL : Predicted Value= `36524.03761290736` , Actual Value= `38415.10600000001`
* ISL : Predicted Value= `4252.446436118835` , Actual Value= `2118.754`
* ISR : Predicted Value= `54632.6173583028` , Actual Value=` 53577.379`
* ITA : Predicted Value= `443797.29175289616` , Actual Value= `443602.78`
* KAZ : Predicted Value= `173687.20804492` , Actual Value= `173398.19341176475`
KOR : Predicted Value= `401640.1529996532` , Actual Value= `401974.22399999993`

We see that the error is very less and also as we did not mandate the use of each group, hence Climate does not have any actiavtion of 1. We have two optimal series from the Size of Economy group only.

# **Discussion About the Model**
---

The Objective function is the sum of square of errors and our goal here is to minimize the errors satisfying the given constraints. 

We conducted multiple experiments and we conclude as below,
* In addition to the lower and upper bound constraints of -100 and 100, other constraints such as at least one series per group and at most 4 series code per group gave us an optimal solution. The predicted values were pretty close to our actual data. 
* However, with the increase in prediction variables from 4 to 5, helped us to get much better results and closer to the actual data.
* We also tried, reducing number of parameters from 4 to 3 to see how the result varies and noticed it provided an infeasible solution as all 4 groups cannot have atleast 1 series code selected as the atmost value is 3. 
* We then tried, modifying our constraints from ac>=1 to ac<=1 as one of the group was not going to have a series code selected. This helped us to achieve our optimal solution for 3 parameters.
* By removing the constraints for our initial model with 4 parameters, we definitely see better results and the errors are much less and closer to actual value.
* We also did not observe any changes when we tried to rerun the solver results. 

# **Conclusion**
---

*	Since real-world data can be dirty, this project taught us that data cleaning and preparation are very important to us to extract important information from our raw data before doing optimization modeling. The time-series data is a good example. There are many null values across 21 years of data.
*	There are a lot of variables and useful information in our master data frame, we learned that we have to be detail-oriented to each column, then extract that useful information as new variables that can be further used in our optimization modeling.
*	When we were structuring our optimization model, we encountered different issues for setting up the constraints. We noticed the importance of setting up the constraints as the model would take a long time to run or show infeasible. The constraints have to be set in the right way.
*	For the modeling part, we built a model based on the project requirement and some models for experiments. These results told us that we should think outside the box and try different constraints to get a better result.
* According to the results in our model, we found that road conditions, climate, economy, and population are important indicators to predict CO2 emissions. By using five features in our model provides more accurate results, we could use more features in the model to get more information such as major industries and the proportion of the number of cars per household for future research. It may provide an in-depth exploration and analysis to predict CO2 emission.